# Week 3 Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

# PART 1

>## *For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.*

>*Start by creating a new Notebook for this assignment.*

>*Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:*

>*To create the above dataframe:*

>*The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood*

>*Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.*

>*More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.*

>*If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.*

>*Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.*

>*In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.*

>*Submit a link to your Notebook on your Github repository. *

In [54]:
#!pip install beautifulsoup4 #install beautifulsoup https://beautiful-soup-4.readthedocs.io/en/latest/
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [55]:
#Get url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text

## Scrape data from url using BeautifulSoup

In [56]:
Toronto_data = BeautifulSoup(source, 'xml')

In [57]:
table=Toronto_data.find('table')

In [58]:
column_names=['Postalcode','Borough','Neighbourhood']
df = pd.DataFrame(columns=column_names)

In [59]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [60]:
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Data Cleaning

In [75]:
#Remove not assigned Borough cells
df=df[df['Borough']!='Not assigned']
df.head()

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [76]:
temp=df.groupby('Postalcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
temp=temp.reset_index(drop=False)
temp.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)

merge = pd.merge(df, temp, on='Postalcode')
merge.drop(['Neighbourhood'],axis=1,inplace=True)
merge.drop_duplicates(inplace=True)
merge.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)
merge.head()

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [77]:
#merge.axes

In [78]:
merge.shape

(103, 3)

# PART 2

In [79]:
!pip install geocoder

In [95]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [85]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [98]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, merge, on='Postalcode')
geodata=geo_merged[['Postalcode','Borough','Neighbourhood','Latitude','Longitude']]

geodata.head(20)

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
